##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 모델 저장과 복원

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

모델 진행 상황은 훈련 중 및 훈련 후에 저장할 수 있습니다. 즉, 모델이 중단된 위치에서 다시 시작하고 긴 훈련 시간을 피할 수 있습니다. 저장은 또한 모델을 공유할 수 있고 다른 사람들이 작업을 다시 만들 수 있음을 의미합니다. 연구 모델 및 기술을 게시할 때 대부분의 머신러닝 실무자는 다음을 공유합니다.

- 모델을 만드는 코드
- 모델의 훈련된 가중치 또는 파라미터

이런 데이터를 공유하면 다른 사람들이 모델의 작동 방식을 이해하고 새로운 데이터로 모델을 실험하는데 도움이 됩니다.

주의: TensorFlow 모델은 코드이며 신뢰할 수 없는 코드에 주의하는 것이 중요합니다. 자세한 내용은 [TensorFlow 안전하게 사용하기](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md)를 참조하세요.

### 저장 방식

사용 중인 API에 따라 TensorFlow 모델을 저장하는 다양한 방법이 있습니다. 이 가이드에서는 TensorFlow에서 모델을 빌드하고 훈련하기 위해 고수준 API인 [tf.keras](https://www.tensorflow.org/guide/keras)를 사용합니다. 다른 접근 방식에 대해서는 TensorFlow [저장 및 복원](https://www.tensorflow.org/guide/saved_model) 가이드 또는 [즉시 실행 저장](https://www.tensorflow.org/guide/eager#object-based_saving)을 참조하세요.

## 설정

### 설치와 임포트

필요한 라이브러리를 설치하고 텐서플로를 임포트(import)합니다:

In [3]:
!pip install pyyaml h5py  # HDF5 포맷으로 모델을 저장하기 위해서 필요합니다

You should consider upgrading via the '/tmpfs/src/tf_docs_env/bin/python -m pip install --upgrade pip' command.


In [4]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.6.0


### 예제 데이터셋 받기

[MNIST 데이터셋](http://yann.lecun.com/exdb/mnist/)으로 모델을 훈련하여 가중치를 저장하는 예제를 만들어 보겠습니다. 모델 실행 속도를 빠르게 하기 위해 샘플에서 처음 1,000개만 사용겠습니다:

In [5]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 모델 정의

먼저 간단한 모델을 하나 만들어 보죠.

In [6]:
# 간단한 Sequential 모델을 정의합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 훈련하는 동안 체크포인트 저장하기

훈련된 모델을 다시 훈련할 필요 없이 사용하거나 훈련 과정이 중단된 경우 중단한 부분에서 훈련을 다시 시작할 수 있습니다. `tf.keras.callbacks.ModelCheckpoint` 콜백을 사용하면 훈련 *도중* 또는 훈련 *종료 시* 모델을 지속적으로 저장할 수 있습니다.

### 체크포인트 콜백 사용하기

훈련하는 동안 가중치를 저장하기 위해 `ModelCheckpoint` 콜백을 만들어 보죠:

In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10


 1/32 [..............................] - ETA: 20s - loss: 2.3014 - accuracy: 0.0938

28/32 [=========================>....] - ETA: 0s - loss: 1.1929 - accuracy: 0.6529 

32/32 [==============================] - 1s 7ms/step - loss: 1.1272 - accuracy: 0.6720 - val_loss: 0.7001 - val_accuracy: 0.7880



Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10


...


Epoch 00009: saving model to training_1/cp.ckpt


Epoch 10/10
 1/32 [..............................] - ETA: 0s - loss: 0.0690 - accuracy: 1.0000

29/32 [==========================>...] - ETA: 0s - loss: 0.0388 - accuracy: 0.9978

32/32 [==============================] - 0s 4ms/step - loss: 0.0405 - accuracy: 0.9980 - val_loss: 0.4204 - val_accuracy: 0.8600



Epoch 00010: saving model to training_1/cp.ckpt


이 코드는 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다:

In [8]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'cp.ckpt.data-00000-of-00001', 'checkpoint']

두 모델이 동일한 아키텍처를 공유하기만 한다면 두 모델 간에 가중치를 공유할 수 있습니다. 따라서 가중치 전용에서 모델을 복원할 때 원래 모델과 동일한 아키텍처로 모델을 만든 다음 가중치를 설정합니다.

이제 훈련되지 않은 새로운 모델을 다시 빌드하고 테스트 세트에서 평가합니다. 훈련되지 않은 모델은 확률 수준(~10% 정확도)에서 수행됩니다.

In [9]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.2990 - accuracy: 0.1190


훈련되지 않은 모델의 정확도: 11.90%


체크포인트에서 가중치를 로드하고 다시 평가해 보죠:

In [10]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4204 - accuracy: 0.8600


복원된 모델의 정확도: 86.00%


### 체크포인트 콜백 매개변수

이 콜백 함수는 몇 가지 매개변수를 제공합니다. 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있습니다.

새로운 모델을 훈련하고 다섯 번의 에포크마다 고유한 이름으로 체크포인트를 저장해 보겠습니다:

In [11]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt



Epoch 00010: saving model to training_2/cp-0010.ckpt



Epoch 00015: saving model to training_2/cp-0015.ckpt



Epoch 00020: saving model to training_2/cp-0020.ckpt



Epoch 00025: saving model to training_2/cp-0025.ckpt



Epoch 00030: saving model to training_2/cp-0030.ckpt



Epoch 00035: saving model to training_2/cp-0035.ckpt



Epoch 00040: saving model to training_2/cp-0040.ckpt



Epoch 00045: saving model to training_2/cp-0045.ckpt



Epoch 00050: saving model to training_2/cp-0050.ckpt


만들어진 체크포인트를 확인해 보고 마지막 체크포인트를 선택해 보겠습니다:

In [12]:
os.listdir(checkpoint_dir)

['cp-0050.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0010.ckpt.data-00000-of-00001']

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

참고: 기본 TensorFlow 형식은 가장 최근의 체크포인트 5개만 저장합니다.

모델을 초기화하고 최근 체크포인트를 로드하여 테스트해 보겠습니다:

In [14]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4838 - accuracy: 0.8770


복원된 모델의 정확도: 87.70%


## 이 파일들은 무엇인가요?

위의 코드는 이진 형식의 훈련된 가중치만 포함하는 [체크포인트](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables) 형식의 파일 모음에 가중치를 저장합니다. 체크포인트에는 다음이 포함됩니다.

- 모델의 가중치를 포함하는 하나 이상의 샤드
- 어떤 가중치가 어떤 샤드에 저장되어 있는지 나타내는 인덱스 파일

단일 머신에서 모델을 훈련하는 경우 접미사가 `.data-00000-of-00001`인 하나의 샤드를 갖게 됩니다.

## 수동으로 가중치 저장하기

`Model.save_weights` 메서드를 사용하여 수동으로 가중치를 저장합니다. 기본적으로 `tf.keras`, 특히  `save_weights`는 `.ckpt` 확장자가 있는 TensorFlow [체크포인트](../../guide/checkpoint.ipynb) 형식을 사용합니다(`.h5` 확장자를 사용하여 [HDF5](https://js.tensorflow.org/tutorials/import-keras.html)에 저장하는 내용은 [모델 저장 및 직렬화](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) 가이드에서 다룸).

In [15]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4838 - accuracy: 0.8770


복원된 모델의 정확도: 87.70%


## 전체 모델 저장하기

[model.save](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) 메서드를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장합니다. 모델을 저장하기 때문에 원본 파이썬 코드*가 없어도 사용할 수 있습니다. 옵티마이저 상태가 복원되므로 정확히 중지한 시점에서 다시 훈련을 시작할 수 있습니다.

전체 모델은 두 가지 다른 파일 형식(`SavedModel` 및 `HDF5`)으로 저장할 수 있습니다. TensorFlow `SavedModel` 형식은 TF2.x의 기본 파일 형식입니다. 그러나 모델을 `HDF5` 형식으로 저장할 수 있습니다. 전체 모델을 두 가지 파일 형식으로 저장하는 방법에 대한 자세한 내용은 아래에 설명되어 있습니다.

전체 모델을 저장하는 기능은 매우 유용합니다. TensorFlow.js로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있습니다([Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html), [HDF5](https://js.tensorflow.org/tutorials/import-keras.html)). 또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행할 수 있습니다([Saved Model](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_), [HDF5](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_)).

* 사용자 정의 객체(예를 들면 상속으로 만든 클래스나 층)는 저장하고 로드하는데 특별한 주의가 필요합니다. 아래 **사용자 정의 객체 저장하기** 섹션을 참고하세요. 

### SavedModel 포맷

SavedModel 형식은 모델을 직렬화하는 또 다른 방법입니다. 이 형식으로 저장된 모델은 `tf.keras.models.load_model`을 사용하여 복원할 수 있으며 TensorFlow Serving과 호환됩니다. [SavedModel 가이드](https://www.tensorflow.org/guide/saved_model)에 SavedModel을 제공/검사하는 방법이 자세히 설명되어 있습니다. 아래 섹션은 모델을 저장하고 복원하는 단계를 보여줍니다.

In [16]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5


 1/32 [..............................] - ETA: 7s - loss: 2.3484 - accuracy: 0.1875

29/32 [==========================>...] - ETA: 0s - loss: 1.2297 - accuracy: 0.6347

32/32 [==============================] - 0s 2ms/step - loss: 1.1855 - accuracy: 0.6510


Epoch 2/5
 1/32 [..............................] - ETA: 0s - loss: 0.5534 - accuracy: 0.7812

...

Epoch 5/5
 1/32 [..............................] - ETA: 0s - loss: 0.2513 - accuracy: 0.9062

30/32 [===========================>..] - ETA: 0s - loss: 0.1571 - accuracy: 0.9625

32/32 [==============================] - 0s 2ms/step - loss: 0.1576 - accuracy: 0.9630


2021-08-25 20:50:57.975912: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/my_model/assets


SavedModel 형식은 protobuf 바이너리와 TensorFlow 체크포인트를 포함하는 디렉토리입니다. 저장된 모델 디렉토리를 검사합니다.

In [17]:
# my_model 디렉토리
!ls saved_model

# assests 폴더, saved_model.pb, variables 폴더
!ls saved_model/my_model

my_model


assets	keras_metadata.pb  saved_model.pb  variables


저장된 모델로부터 새로운 케라스 모델을 로드합니다:

In [18]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있습니다. 이 모델을 평가하고 예측에 사용해 보죠:

In [19]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4343 - accuracy: 0.8660


복원된 모델의 정확도: 86.60%


(1000, 10)


### HDF5 파일로 저장하기

케라스는 [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) 표준을 따르는 기본 저장 포맷을 제공합니다. 

In [20]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5') 

Epoch 1/5


 1/32 [..............................] - ETA: 7s - loss: 2.3217 - accuracy: 0.0625

29/32 [==========================>...] - ETA: 0s - loss: 1.1929 - accuracy: 0.6692

32/32 [==============================] - 0s 2ms/step - loss: 1.1460 - accuracy: 0.6840


Epoch 2/5
 1/32 [..............................] - ETA: 0s - loss: 0.4662 - accuracy: 0.9062

...

29/32 [==========================>...] - ETA: 0s - loss: 0.1959 - accuracy: 0.9526

32/32 [==============================] - 0s 2ms/step - loss: 0.1997 - accuracy: 0.9520


Epoch 5/5
 1/32 [..............................] - ETA: 0s - loss: 0.1610 - accuracy: 0.9375

29/32 [==========================>...] - ETA: 0s - loss: 0.1462 - accuracy: 0.9709

32/32 [==============================] - 0s 2ms/step - loss: 0.1450 - accuracy: 0.9710


이제 이 파일로부터 모델을 다시 만들어 보죠:

In [21]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


정확도를 확인해 보겠습니다:

In [22]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4581 - accuracy: 0.8580


복원된 모델의 정확도: 85.80%


Keras는 아키텍처를 검사하여 모델을 저장합니다. 이 기술은 모든 내용을 저장합니다.

- 가중치 값
- 모델 구조
- 모델의 훈련 구성(`.compile()` 메서드에 전달하는 내용)
- 존재하는 옵티마이저와 그 상태(훈련을 중단한 곳에서 다시 시작할 수 있게 해줌)

체크포인트가 호환되지 않기 때문에 케라스는 v1.x 옵티마이저(`tf.compat.v1.train`)를 저장할 수 없습니다. v1.x 옵티마이저를 사용하려면 로드한 후에 모델을 다시 컴파일해야 합니다. 따라서 옵티마이저의 상태를 잃게 됩니다.


### 사용자 정의 객체

SavedModel 형식을 사용하는 경우, 이 섹션을 건너뛸 수 있습니다. HDF5와 SavedModel의 주요 차이점은 HDF5는 객체 구성을 사용하여 모델 아키텍처를 저장하는 반면, SavedModel은 실행 그래프를 저장한다는 것입니다. 따라서 SavedModel은 원본 코드 없이도 서브클래싱된 모델 및 사용자 지정 레이어와 같은 사용자 지정 객체를 저장할 수 있습니다.

사용자 정의 객체를 HDF5로 저장하려면 다음 과정을 따르세요:

1. 이 객체에 `get_config` 메서드를 정의하고 선택적으로 `from_config` 클래스 메서드를 정의합니다.
    - `get_config(self)`는 객체를 다시 생성하기 위해 필요한 JSON 직렬화된 매개변수 딕셔너리를 반환합니다.
    - `from_config(cls, config)`는 `get_config`에서 반환된 설정을 사용해 새로운 객체를 만듭니다. 기본적으로 이 함수는 이 설정을 초기화 메서드의 매개변수로 사용합니다(`return cls(**config)`).
2. 모델을 로드할 때 이 객체를 `custom_objects` 매개변수로 전달합니다. 문자열 클래스 이름과 파이썬 클래스를 매핑한 딕서너리를 매개변수로 제공해야 합니다. 예를 들면 `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

사용자 정의 객체와 `get_config`에 관한 예제를 보려면 [Writing layers and models from scratch](https://www.tensorflow.org/guide/keras/custom_layers_and_models) 튜토리얼을 참고하세요.
